get 10000 random images

benign = 393 and malignant = 10000

train with alexnet

result

Epoch 01 - Train: 0.2200, 96.22% | Val: 0.2350, 96.20%

Epoch 02 - Train: 0.1892, 96.22% | Val: 0.1617, 96.20%

Epoch 03 - Train: 0.3639, 95.66% | Val: 0.1620, 96.20%

Epoch 04 - Train: 0.1716, 96.22% | Val: 0.1621, 96.20%

Epoch 05 - Train: 0.4603, 95.91% | Val: 0.1622, 96.20%

Epoch 06 - Train: 0.1651, 96.22% | Val: 0.1617, 96.20%

Epoch 07 - Train: 0.1628, 96.22% | Val: 0.1634, 96.20%

Epoch 08 - Train: 0.1625, 96.22% | Val: 0.1650, 96.20%

Epoch 09 - Train: 5.3418, 95.02% | Val: 0.1747, 96.20%

Epoch 10 - Train: 0.1726, 96.14% | Val: 0.1626, 96.20%

In [33]:
import os
import pandas as pd
import shutil

# === 1. Load metadata CSV and prepare paths ===
meta_dir = '/kaggle/input/isic-2024-challenge/'
image_dir = os.path.join(meta_dir, 'train-image/image')
csv_path = os.path.join(meta_dir, 'train-metadata.csv')

# Read the metadata
df = pd.read_csv(csv_path)

# Add image filename column
df['filename'] = df['isic_id'].astype(str) + '.jpg'

print("Total images in dataset:", len(df))

/tmp/ipykernel_35/91138490.py:11: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


Total images in dataset: 401059


In [34]:
# filter malignant (target == 1)
df_positive = df[df["target"] == 1]

# filter benign (target == 0), randomly sample 10,000 rows
df_negative = df[df["target"] == 0].sample(n=10000, random_state=42)

# combine both positive and negative cases
df_balanced = pd.concat([df_positive, df_negative]).reset_index(drop=True)

# shuffle the data
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# save to a new CSV
csv_output_path = "/kaggle/working/train_created.csv"
df_balanced.to_csv(csv_output_path, index=False)

print("Balanced CSV saved to:", csv_output_path)
print("Total balanced samples:", len(df_balanced))
print("Malignant:", (df_balanced['target'] == 1).sum(), "| Benign:", (df_balanced['target'] == 0).sum())

Balanced CSV saved to: /kaggle/working/train_created.csv
Total balanced samples: 10393
Malignant: 393 | Benign: 10000


In [35]:
output_dir = '/kaggle/working/sampled_dataset'
malignant_out = os.path.join(output_dir, 'malignant')
benign_out = os.path.join(output_dir, 'benign')
os.makedirs(malignant_out, exist_ok=True)
os.makedirs(benign_out, exist_ok=True)

# Copy images by class
def copy_images(df_subset, label_value, dest_folder):
    df_filtered = df_subset[df_subset["target"] == label_value]
    for _, row in df_filtered.iterrows():
        src_path = os.path.join(image_dir, row['filename'])
        dst_path = os.path.join(dest_folder, row['filename'])
        if os.path.exists(src_path):
            shutil.copy(src_path, dst_path)

# Copy malignant (target == 1) and benign (target == 0)
copy_images(df_balanced, 1, malignant_out)
copy_images(df_balanced, 0, benign_out)

In [36]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os
from sklearn.model_selection import train_test_split
import time
import pickle

In [37]:
image_paths, labels = [], []
for img_file in os.listdir(malignant_out):
    if img_file.endswith('.jpg'):
        image_paths.append(os.path.join(malignant_out, img_file))
        labels.append(1)
for img_file in os.listdir(benign_out):
    if img_file.endswith('.jpg'):
        image_paths.append(os.path.join(benign_out, img_file))
        labels.append(0)

train_paths, val_paths, train_labels, val_labels = train_test_split(
    image_paths, labels, test_size=0.2, stratify=labels, random_state=42
)

In [38]:
class ISICDataset(Dataset):
    def __init__(self, paths, labels, transform):
        self.paths = paths
        self.labels = labels
        self.transform = transform
    def __len__(self): return len(self.paths)
    def __getitem__(self, idx):
        img = Image.open(self.paths[idx]).convert('RGB')
        return self.transform(img), self.labels[idx]

train_loader = DataLoader(ISICDataset(train_paths, train_labels, train_transform), batch_size=16, shuffle=True)
val_loader = DataLoader(ISICDataset(val_paths, val_labels, val_transform), batch_size=16, shuffle=False)

In [39]:
class AlexNet(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, 11, 4, 2), nn.ReLU(inplace=True), nn.MaxPool2d(3, 2),
            nn.Conv2d(64, 192, 5, padding=2), nn.ReLU(inplace=True), nn.MaxPool2d(3, 2),
            nn.Conv2d(192, 384, 3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, 3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, 3, padding=1), nn.ReLU(inplace=True), nn.MaxPool2d(3, 2)
        )
        self.classifier = nn.Sequential(
            nn.Dropout(0.5), nn.Linear(256 * 6 * 6, 4096), nn.ReLU(inplace=True),
            nn.Dropout(0.5), nn.Linear(4096, 4096), nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes)
        )
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        return self.classifier(x)

In [40]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = AlexNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)

import torch
print(torch.cuda.is_available())
print("Using device:", device)

True
Using device: cuda


In [41]:
def run_epoch(loader, train):
    model.train() if train else model.eval()
    total, correct, loss_sum = 0, 0, 0
    with torch.set_grad_enabled(train):
        for imgs, labels in loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            if train:
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            _, preds = outputs.max(1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            loss_sum += loss.item()
    return loss_sum / len(loader), 100. * correct / total

In [42]:
epochs = 10
train_losses, train_accs = [], []
val_losses, val_accs = [], []

start = time.time()

for epoch in range(epochs):
    train_loss, train_acc = run_epoch(train_loader, True)
    val_loss, val_acc = run_epoch(val_loader, False)
    train_losses.append(train_loss)
    train_accs.append(train_acc)
    val_losses.append(val_loss)
    val_accs.append(val_acc)
    print(f"Epoch {epoch+1:02d} - Train: {train_loss:.4f}, {train_acc:.2f}% | Val: {val_loss:.4f}, {val_acc:.2f}%")

# === Save Results ===
torch.save(model.state_dict(), '/kaggle/working/alexnet_model.pth')
with open('/kaggle/working/alexnet_results.pkl', 'wb') as f:
    pickle.dump({
        'train_losses': train_losses,
        'val_losses': val_losses,
        'train_accs': train_accs,
        'val_accs': val_accs,
        'epochs': epochs,
        'best_val_acc': max(val_accs),
        'training_time': time.time() - start
    }, f)

Epoch 01 - Train: 0.2200, 96.22% | Val: 0.2350, 96.20%
Epoch 02 - Train: 0.1892, 96.22% | Val: 0.1617, 96.20%
Epoch 03 - Train: 0.3639, 95.66% | Val: 0.1620, 96.20%
Epoch 04 - Train: 0.1716, 96.22% | Val: 0.1621, 96.20%
Epoch 05 - Train: 0.4603, 95.91% | Val: 0.1622, 96.20%
Epoch 06 - Train: 0.1651, 96.22% | Val: 0.1617, 96.20%
Epoch 07 - Train: 0.1628, 96.22% | Val: 0.1634, 96.20%
Epoch 08 - Train: 0.1625, 96.22% | Val: 0.1650, 96.20%
Epoch 09 - Train: 5.3418, 95.02% | Val: 0.1747, 96.20%
Epoch 10 - Train: 0.1726, 96.14% | Val: 0.1626, 96.20%
